In [1]:
import pandas as pd
import time
import joblib
import matplotlib.pyplot as plt
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import DistanceMetric
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
df = pd.read_csv(r'D:\JupyterNotebook\DataProcess\new_raw_data.csv')
df.columns = ['text', 'label']
df

,text,label
0,- Satresnarkoba Polres Anambas bersama Polres...,4
1,"- Berbagai usaha dilakukan, mulai dari olahra...",2
2,"- RG (22), pemuda yang nekat menusuk Z (16), ...",4
3,- Para suporter Thailand sempat menyapa dan m...,0
4,"- , keenam Kualifikasi Piala Eropa 2020 mempe...",0
...,...,...
3248,"- Kondisi S (48), guru SDN , 07 Pagi yang me...",4
3249,"- Artis peran dan penyanyi , menjadi salah s...",1
3250,"- Manfaat , untuk perawatan kulit sudah tak ...",2
3251,"- Artis peran , mengaku sangat terganggu den...",1


In [13]:
df = pd.read_pickle(r'D:\JupyterNotebook\DataProcess\preprocess_SWsastrawi_stemming.pkl')
dfNo = pd.read_pickle(r'D:\JupyterNotebook\DataProcess\preprocess_SWsastrawi_nostemming.pkl')

In [8]:
kf = KFold(n_splits=5)
i = 1
for train_index, test_index in kf.split(df):
#     print(test_index)
#     print(train_index[0],"-",train_index[len(train_index)-1])
    print(test_index[0],"+",test_index[len(test_index)-1])
    X_train = df['text'].iloc[train_index[0]:train_index[len(train_index)-1]]
    X_test = df['text'].iloc[test_index[0]:test_index[len(test_index)-1]]
    y_train = df['label'].iloc[train_index[0]:train_index[len(train_index)-1]].astype('int')
    y_test = df['label'].iloc[test_index[0]:test_index[len(test_index)-1]].astype('int')
    
#     vectorizer = TfidfVectorizer()
#     X_train = vectorizer.fit_transform(X_train)
#     X_test = vectorizer.transform(X_test)
    counter = CountVectorizer(min_df=3,
                              dtype=np.double)
    counter.fit(X_train)
    X_train = counter.transform(X_train)
    X_test = counter.transform(X_test)
    
    transformer = TfidfTransformer(norm='l2', sublinear_tf=True)
    X_train = transformer.fit_transform(X_train)
    X_test = transformer.transform(X_test)
    
    X_train,X_test = select_features(X_train, y_train, X_test)
    
    modelknn2 = KNeighborsClassifier(n_neighbors=13, weights='uniform', metric='euclidean')
    modelknn2.fit(X_train,y_train)
    y_pred = modelknn2.predict(X_test)
#     print(classification_report(y_test,y_pred))
    result = modelknn2.score(X_test, y_test)
    print("Fold ke-",i,"=",result)
    print("----------------")
    i+=1

0 + 650


ValueError: invalid literal for int() with base 10: 'Entertainment'

In [3]:
def split(strt,fnsh,df):
    X_test = df['text'].iloc[strt:fnsh]
    y_test = df['label'].iloc[strt:fnsh].astype('int')
    df.drop(df.index[strt:fnsh])
    X_train = df['text']
    y_train = df['label'].astype('int')    
    return X_train, X_test, y_train, y_test

In [4]:
def select_features(train_X, train_y, test_X):
#     total fitur 7343
# 2004
# 1802 nltk
# 2599 sastrawi 1967
    selector = SelectKBest(chi2, k=1967)
    selector.fit(train_X, train_y)
#     new = selector.fit_transform(train_X, train_y)
#     print(new.shape)
    train_X = selector.transform(train_X)
    test_X = selector.transform(test_X)
    return train_X, test_X

In [5]:
def classifier(df,distance_name,k_neighbor):
    print("Nama distance=",distance_name)
    print("Jumlah K tetangga=",k_neighbor)
    spl = [0,36,72,108,144,180,216,252,288,324,360]
    tempspl = []
    i = 0
    akurasimax=0
    while (i<len(spl)-1):
        if len(spl)-2 != len(tempspl):
            X_train, X_test, y_train, y_test = split(spl[i],spl[i+2],df)
            tempspl.append(spl[i])
    #   ketika array[i+2] == null (looping terakhir)
        elif (len(spl)-len(tempspl)) == 2:
    #       split last and first
            X_test1 = df['text'].iloc[spl[i]:spl[i+1]]
            y_test1 = df['label'].iloc[spl[i]:spl[i+1]].astype('int')
            X_test2 = df['text'].iloc[tempspl[0]:tempspl[1]]
            y_test2 = df['label'].iloc[tempspl[0]:tempspl[1]].astype('int')
    #       concat last and first
            X_test = pd.concat([X_test1, X_test2], axis=0)
            y_test = pd.concat([y_test1, y_test2], axis=0)
    #       split except last and first
            X_train = df['text'].iloc[tempspl[1]:tempspl[len(tempspl)-1]]
            y_train = df['label'].iloc[tempspl[1]:tempspl[len(tempspl)-1]].astype('int')
        
        vectorizer = TfidfVectorizer(min_df=3, dtype=np.double, norm='l2', sublinear_tf=True)
        X_train = vectorizer.fit_transform(X_train)
        X_test = vectorizer.transform(X_test)

#         counter = CountVectorizer(min_df=3,
#                               dtype=np.double)
#         counter.fit(X_train)
#         X_train = counter.transform(X_train)
#         X_test = counter.transform(X_test)

#         transformer = TfidfTransformer(norm='l2', sublinear_tf=True)
#         X_train = transformer.fit_transform(X_train)
#         X_test = transformer.transform(X_test)
        
#         X_train,X_test = select_features(X_train, y_train, X_test)

        modelknn = KNeighborsClassifier(n_neighbors=k_neighbor, weights='uniform', metric=distance_name)
        modelknn.fit(X_train,y_train)
        y_pred = modelknn.predict(X_test)
#         print(classification_report(y_test,y_pred))
        result = modelknn.score(X_test, y_test)
        print("Fold ke-",i+1,"=",result)
        if (result >= akurasimax):
            akurasimax = result
            foldke = i+1
        i+=1
    print("Akurasi terbaik adalah",akurasimax,"pada fold ke-",foldke)
    print("------------------------------------")
    return akurasimax

In [22]:
%%time
# %%capture cap --no-stderr
start_time = time.time()
dfcount = [df,dfNo]
distancecount = ['euclidean','manhattan','minkowski']
kcount = [1,3,5,7,9,11,13,15]
for x in range(2):
    arrd1 = []
    arrd2 = []
    arrd3 = []
    for y in range(len(distancecount)):
        maxresult = []
        for z in range(len(kcount)):
            if x==0:
                print('Dengan Stemming')
            else:
                print('Tanpa Stemming')
            tempmax = classifier(dfcount[x],distancecount[y],kcount[z])
            maxresult.append(tempmax)
            if y==0:
                arrd1.append(tempmax)
            elif y==1:
                arrd2.append(tempmax)
            elif y==2:
                arrd3.append(tempmax)
#         tapi fold yang dipakai beda beda
    plt.plot(kcount, arrd1, label = "euclidean")
    plt.plot(kcount, arrd2, label = "manhattan")
    plt.plot(kcount, arrd3, label = "minkowski")
    plt.xticks(kcount)
    if x==0:
        plt.title("Stemming")
    else:
        plt.title("Tidak Stemming")
    plt.xlabel('Fold')
    plt.ylabel('Akurasi')
    plt.legend()
    plt.pause(0.05)
plt.show
print("Lama Proses %s seconds" % (time.time() - start_time))
# with open('5000_fitur_SWnltkv2.txt', 'w') as f:
#     f.write(cap.stdout)

Dengan Stemming
Nama distance= euclidean
Jumlah K tetangga= 1
Fold ke- 1 = 1.0
Fold ke- 2 = 1.0
Fold ke- 3 = 1.0
Fold ke- 4 = 1.0
Fold ke- 5 = 1.0
Fold ke- 6 = 1.0
Fold ke- 7 = 1.0
Fold ke- 8 = 1.0
Fold ke- 9 = 1.0
Fold ke- 10 = 0.125
Akurasi terbaik adalah 1.0 pada fold ke- 9
------------------------------------
Dengan Stemming
Nama distance= euclidean
Jumlah K tetangga= 3
Fold ke- 1 = 0.8333333333333334
Fold ke- 2 = 0.8611111111111112
Fold ke- 3 = 0.7916666666666666
Fold ke- 4 = 0.8194444444444444
Fold ke- 5 = 0.8194444444444444
Fold ke- 6 = 0.7916666666666666
Fold ke- 7 = 0.875
Fold ke- 8 = 0.9166666666666666
Fold ke- 9 = 0.9027777777777778
Fold ke- 10 = 0.6111111111111112
Akurasi terbaik adalah 0.9166666666666666 pada fold ke- 8
------------------------------------
Dengan Stemming
Nama distance= euclidean
Jumlah K tetangga= 5
Fold ke- 1 = 0.8333333333333334
Fold ke- 2 = 0.8194444444444444
Fold ke- 3 = 0.8194444444444444
Fold ke- 4 = 0.8472222222222222
Fold ke- 5 = 0.819444444444444

KeyboardInterrupt: 

# try--

In [ ]:
X_train = df['text'].iloc[72:]
X_test = df['text'].iloc[:72]
y_train = df['label'].iloc[72:].astype('int')
y_test = df['label'].iloc[:72].astype('int')

In [26]:
X = df['text']
y = df['label'].astype('int')
# X = dfNo['text']
# y = dfNo['label'].astype('int')

In [27]:
# Pembentukan tf-idf untuk pembobotan
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X)

# X_train = vectorizer.fit_transform(X_train)
# X_test = vectorizer.transform(X_test)

In [9]:
print(X_train.shape)

NameError: name 'X_train' is not defined

In [28]:
print(vectorizer.vocabulary_)
# print(len(vectorizer.get_feature_names()))

{'satresnarkoba': 26492, 'polres': 24169, 'anambas': 1857, 'bersama': 4100, 'tanjungpinang': 29222, 'mengamankan': 18326, 'suria': 28767, 'adik': 1227, 'kandung': 13041, 'wali': 31594, 'kota': 15072, 'syharul': 28951, 'dan': 6133, 'hendra': 11033, 'alias': 1644, 'indra': 11748, 'jepang': 12535, 'di': 6539, 'salah': 26202, 'satu': 26498, 'rumah': 25975, 'jalan': 12313, 'tanjung': 29219, 'unggat': 31093, 'kecamatan': 13452, 'timur': 30316, 'kepulauan': 14060, 'riau': 25587, 'penangkapan': 22503, 'keduanya': 13534, 'merupakan': 19634, 'hasil': 10911, 'dari': 6192, 'pengembangan': 22774, 'kasus': 13264, 'narkoba': 20531, 'yang': 32025, 'tengah': 29529, 'diungkap': 8409, 'kasubag': 13259, 'humas': 11394, 'ipda': 12010, 'zulmi': 32335, 'mengatakan': 18394, 'ini': 11823, 'sedang': 26695, 'ditangani': 8213, 'satresnatkoba': 26493, 'dalam': 6099, 'penyelidikan': 23136, 'diketahui': 7275, 'bahwa': 2747, 'sabu': 26102, 'beredar': 3579, 'berasal': 3421, 'diduga': 6930, 'sebagai': 26615, 'pemasok':

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [30]:
%%time
modelknn2 = KNeighborsClassifier(n_neighbors=11, weights='uniform', metric='euclidean')
modelknn2.fit(X_train,y_train)

Wall time: 5.98 ms


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean',
                     metric_params=None, n_jobs=None, n_neighbors=11, p=2,
                     weights='uniform')

In [ ]:
filename = 'D:\JupyterNotebook\Model\model_nostemming_173_splitmanual.sav'
joblib.dump(modelknn2, filename)

In [ ]:
filename = 'D:\JupyterNotebook\Model\model_stemming.sav'
loaded_model = joblib.load(filename)
# result = loaded_model.score(X_test, y_test)
# print(result)

In [31]:
y_pred = modelknn2.predict(X_test)
y_pred

array([4, 3, 4, 4, 4, 4, 4, 4, 3, 6, 0, 4, 4, 3, 4, 4, 4, 6, 4, 1, 4, 0,
       4, 4, 4, 4, 4, 4, 4, 4, 8, 4, 4, 4, 4, 0, 4, 4, 4, 3, 4, 4, 4, 0,
       4, 4, 4, 4, 9, 0, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 3, 1, 4, 4, 3,
       4, 4, 0, 4, 3, 1, 4, 4, 1, 4, 0, 4, 4, 4, 1, 4, 4, 0, 4, 4, 4, 3,
       4, 3, 3, 3, 9, 4, 4, 4, 4, 2, 1, 4, 1, 4, 4, 4, 4, 0, 3, 4, 4, 0,
       4, 3, 4, 4, 4, 3, 2, 4, 4, 4, 4, 4, 0, 0, 1, 4, 1, 4, 1, 2, 3, 0,
       9, 4, 4, 4, 4, 4, 4, 1, 4, 4, 4, 4, 4, 4, 1, 4, 4, 0, 4, 4, 4, 0,
       4, 1, 0, 4, 4, 4, 4, 6, 0, 4, 4, 1, 4, 6, 4, 4, 4, 9, 4, 4, 1, 4,
       4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 0, 3, 1, 4, 4, 4, 3, 0, 0, 4, 4, 4,
       4, 4, 0, 4, 4, 4, 4, 0, 4, 4, 1, 4, 4, 4, 0, 5, 4, 1, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 0, 0, 4, 0, 4, 4, 3, 1, 4, 3, 2, 4, 4, 1, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 3, 0, 3, 4, 4, 5,
       0, 4, 4, 4, 4, 4, 0, 4, 4, 3, 4, 1, 4, 0, 4, 4, 3, 4, 2, 4, 4, 4,
       0, 0, 4, 0, 4, 4, 4, 0, 0, 4, 4, 4, 4, 4, 4,

In [ ]:
y_pred = loaded_model.predict(X_test)
y_pred

In [32]:
modelknn2.predict_proba(X_test)

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.09090909],
       [0.        , 0.09090909, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.54545455, ..., 0.27272727, 0.        ,
        0.        ],
       [0.        , 0.90909091, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.27272727, 0.        , 0.        , ..., 0.        , 0.        ,
        0.09090909]])

In [15]:
f1score_, recall_, precision_, cm_ = evaluation(d_test.label, y_pred_sw, label)
print(" f1-score: {}\n precision: {}\n recall: {}".format(f1score, precision, recall))

NameError: name 'evaluation' is not defined

In [33]:
print(confusion_matrix(y_test, y_pred))

[[ 72   0   0   0   6   0   0   0   0   0]
 [  1  34   1   0   8   0   0   0   0   0]
 [  2   1  15   2   4   0   0   0   0   0]
 [  0   0   1  27  15   1   0   0   1   0]
 [  0   4   0   8 358   1   0   0   0   0]
 [  3   0   0   0  11  15   0   0   0   0]
 [  0   0   0   4   4   0   9   0   0   0]
 [  0   1   2   1   7   0   0   3   0   0]
 [  0   0   1   1   0   0   0   0  10   0]
 [  2   0   0   0   7   1   0   0   0   7]]


In [34]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.90      0.92      0.91        78
           1       0.85      0.77      0.81        44
           2       0.75      0.62      0.68        24
           3       0.63      0.60      0.61        45
           4       0.85      0.96      0.91       371
           5       0.83      0.52      0.64        29
           6       1.00      0.53      0.69        17
           7       1.00      0.21      0.35        14
           8       0.91      0.83      0.87        12
           9       1.00      0.41      0.58        17

    accuracy                           0.84       651
   macro avg       0.87      0.64      0.71       651
weighted avg       0.85      0.84      0.83       651



In [18]:
result = modelknn2.score(X_test, y_test)
print(result)

0.8417818740399385


In [35]:
result_class = pd.DataFrame({'Actual Result':y_test, 'Predicted Result':y_pred})
result_class

,Actual Result,Predicted Result
2602,4,4
2603,4,3
2604,4,4
2605,4,4
2606,4,4
...,...,...
3248,4,4
3249,1,1
3250,2,2
3251,1,1


In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(result_class)